In [4]:
# ─── Dependências ───
# pip install python-dotenv requests yfinance pandas

from dotenv import load_dotenv
import os, json, requests, yfinance as yf, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# ─── 1) Carrega a chave FMP do .env ───
load_dotenv()
API_KEY = os.getenv("FMP_API_KEY")
if not API_KEY:
    raise RuntimeError("FMP_API_KEY não encontrada no .env")

def safe_json(resp: requests.Response):
    """Tenta fazer resp.json(), ou dá um erro mostrando os primeiros chars."""
    try:
        return resp.json()
    except json.JSONDecodeError:
        preview = resp.text[:200].replace("\n"," ")
        raise ValueError(f"Resposta não-JSON da URL {resp.url!r}: {preview!r}")

def get_bulk_scores():
    """Busca e parseia o NDJSON de scores-bulk."""
    url = f"https://financialmodelingprep.com/api/v4/scores-bulk?apikey={API_KEY}"
    r = requests.get(url); r.raise_for_status()
    lines = [l for l in r.text.splitlines() if l.strip()]
    scores = []
    for ln in lines:
        try:
            scores.append(json.loads(ln))
        except json.JSONDecodeError:
            # ignora linhas inválidas
            continue
    return scores

# Pré-carrega o bulk para performance
ALL_SCORES = get_bulk_scores()

def get_metrics(symbol: str) -> dict:
    # ── 2.1) Quote básico via FMP
    r1 = requests.get(f"https://financialmodelingprep.com/api/v3/quote/{symbol}?apikey={API_KEY}")
    r1.raise_for_status()
    qlist = safe_json(r1)
    if not isinstance(qlist, list) or not qlist:
        raise ValueError(f"Quote inesperado para {symbol}: {qlist!r}")
    q = qlist[0]

    # Extrai o que vier
    open_price    = q.get("open")
    last_price    = q.get("price")
    variation_pct = q.get("changesPercentage")
    eps           = q.get("eps")
    pe            = q.get("pe")

    # ── 2.2) Target, dividendos e beta via yfinance
    tk   = yf.Ticker(symbol)
    info = tk.info
    target_price   = info.get("targetMeanPrice")
    expected_gain  = (target_price / last_price - 1) * 100 if target_price and last_price else None
    dividend_yield = info.get("dividendYield") * 100 if info.get("dividendYield") else None
    beta           = info.get("beta")

    # ── 2.3) Perfil da empresa via FMP Profile
    r2 = requests.get(f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={API_KEY}")
    r2.raise_for_status()
    plist = safe_json(r2)
    prof = plist[0] if isinstance(plist, list) and plist else {}
    issuer   = prof.get("companyName")
    sector   = prof.get("sector")
    industry = prof.get("industry")

    # ── 2.4) EMAs via histórico 1y do yfinance
    hist = tk.history(period="1y")
    ema20  = hist["Close"].ewm(span=20,  adjust=False).mean().iloc[-1] if not hist.empty else None
    ema200 = hist["Close"].ewm(span=200, adjust=False).mean().iloc[-1] if not hist.empty else None

    # ── 2.5) Scores do bulk pré-carregado
    rec = next((r for r in ALL_SCORES if r.get("symbol")==symbol), {})
    score_pe       = rec.get("scorePe")
    score_gauss    = rec.get("scoreGauss")
    score_ebitda   = rec.get("scoreEbitda")
    score_cashflow = rec.get("scoreCashFlow")
    final_score    = rec.get("finalScore")

    return {
        "Ticker":                symbol,
        "Emissor":               issuer,
        "Setor":                 sector,
        "Indústria":             industry,
        "Preço Abertura":        open_price,
        "Último Preço":          last_price,
        "Variação (%)":          variation_pct,
        "Preço Alvo":            target_price,
        "Ganho Esperado (%)":    round(expected_gain,2) if expected_gain is not None else None,
        "LPA":                   eps,
        "P/L":                   pe,
        "Dividendos (%)":        dividend_yield,
        "Beta":                  beta,
        "Entrada (EMA20)":       round(ema20,2) if ema20 is not None else None,
        "Saída (EMA200)":        round(ema200,2) if ema200 is not None else None,
        "Score P/L":             score_pe,
        "Score Gauss":           score_gauss,
        "Score EBITDA":          score_ebitda,
        "Score Fluxo de Caixa":  score_cashflow,
        "Placar Final":          final_score
    }

# ─── 3) Exemplo de uso ───
if __name__=="__main__":
    tickers = ["AAPL","MSFT","NVDA"]  # troque pela sua lista
    df = pd.DataFrame([get_metrics(t) for t in tickers])
    print(df)


  Ticker                Emissor       Setor                  Indústria  \
0   AAPL             Apple Inc.  Technology       Consumer Electronics   
1   MSFT  Microsoft Corporation  Technology  Software - Infrastructure   
2   NVDA     NVIDIA Corporation  Technology             Semiconductors   

   Preço Abertura  Último Preço  Variação (%)  Preço Alvo  Ganho Esperado (%)  \
0         204.505      202.4400      0.029647   232.63078               14.91   
1         528.190      535.8706      2.243920   601.66724               12.28   
2         175.160      179.3200      3.223580   182.79474                1.94   

     LPA    P/L  Dividendos (%)   Beta  Entrada (EMA20)  Saída (EMA200)  \
0   7.26  27.88            51.0  1.165           208.81          214.29   
1  13.63  39.32            63.0  1.055           511.59          441.78   
2   3.09  58.03             2.0  2.145           170.43          134.98   

  Score P/L Score Gauss Score EBITDA Score Fluxo de Caixa Placar Final  
0   

In [6]:
from dotenv import load_dotenv
import os, requests, yfinance as yf, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 1) Carrega sua chave FMP do .env
load_dotenv()
API_KEY = os.getenv("FMP_API_KEY")
if not API_KEY:
    raise RuntimeError("FMP_API_KEY não encontrada no .env")

def get_metrics(symbol: str) -> dict:
    # ── 2.1) Quote básico via FMP
    r1 = requests.get(f"https://financialmodelingprep.com/api/v3/quote/{symbol}?apikey={API_KEY}")
    r1.raise_for_status()
    q = r1.json()[0]  # sempre retorna uma lista de 1 item

    open_price    = q.get("open")
    last_price    = q.get("price")
    variation_pct = q.get("changesPercentage")
    eps           = q.get("eps")
    pe            = q.get("pe")

    # ── 2.2) Dados Yahoo Finance
    tk   = yf.Ticker(symbol)
    info = tk.info
    target_price   = info.get("targetMeanPrice")
    expected_gain  = (target_price / last_price - 1) * 100 if target_price and last_price else None
    dividend_yield = info.get("dividendYield")
    if dividend_yield is not None:
        dividend_yield *= 100
    beta = info.get("beta")

    # ── 2.3) Perfil da empresa via FMP Profile
    r2 = requests.get(f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={API_KEY}")
    r2.raise_for_status()
    prof_list = r2.json()
    prof = prof_list[0] if prof_list else {}
    issuer   = prof.get("companyName")
    sector   = prof.get("sector")
    industry = prof.get("industry")

    # ── 2.4) EMAs (1 ano de histórico)
    hist = tk.history(period="1y")
    ema20  = hist["Close"].ewm(span=20,  adjust=False).mean().iloc[-1] if not hist.empty else None
    ema200 = hist["Close"].ewm(span=200, adjust=False).mean().iloc[-1] if not hist.empty else None

    # ── 2.5) Scores individuais via /score/{symbol}
    r3 = requests.get(f"https://financialmodelingprep.com/api/v3/score/{symbol}?apikey={API_KEY}")
    r3.raise_for_status()
    score_list = r3.json()
    s = score_list[0] if isinstance(score_list, list) and score_list else {}

    score_pe       = s.get("scorePe")
    score_gauss    = s.get("scoreGauss")
    score_ebitda   = s.get("scoreEbitda")
    score_cashflow = s.get("scoreCashFlow")
    final_score    = s.get("finalScore")

    return {
        "Ticker":               symbol,
        "Emissor":              issuer,
        "Setor":                sector,
        "Indústria":            industry,
        "Preço Abertura":       open_price,
        "Último Preço":         last_price,
        "Variação (%)":         variation_pct,
        "Preço Alvo":           target_price,
        "Ganho Esperado (%)":   round(expected_gain, 2) if expected_gain is not None else None,
        "LPA":                  eps,
        "P/L":                  pe,
        "Dividendos (%)":       round(dividend_yield, 2) if dividend_yield is not None else None,
        "Beta":                 beta,
        "Entrada (EMA20)":      round(ema20, 2) if ema20 is not None else None,
        "Saída (EMA200)":       round(ema200, 2) if ema200 is not None else None,
        "Score P/L":            score_pe,
        "Score Gauss":          score_gauss,
        "Score EBITDA":         score_ebitda,
        "Score Fluxo de Caixa": score_cashflow,
        "Placar Final":         final_score
    }

# ─── Exemplo de uso ───
if __name__ == "__main__":
    tickers = ["AAPL", "MSFT", "NVDA"]
    df = pd.DataFrame([get_metrics(t) for t in tickers])
    print(df)


  Ticker                Emissor       Setor                  Indústria  \
0   AAPL             Apple Inc.  Technology       Consumer Electronics   
1   MSFT  Microsoft Corporation  Technology  Software - Infrastructure   
2   NVDA     NVIDIA Corporation  Technology             Semiconductors   

   Preço Abertura  Último Preço  Variação (%)  Preço Alvo  Ganho Esperado (%)  \
0         204.505      202.8200       0.21741   232.63078               14.70   
1         528.190      535.9100       2.25144   601.66724               12.27   
2         175.160      179.1765       3.14097   182.79474                2.02   

     LPA    P/L  Dividendos (%)   Beta  Entrada (EMA20)  Saída (EMA200)  \
0   7.26  27.94            51.0  1.165           208.85          214.30   
1  13.63  39.32            63.0  1.055           511.60          441.78   
2   3.09  57.99             2.0  2.145           170.42          134.97   

  Score P/L Score Gauss Score EBITDA Score Fluxo de Caixa Placar Final  
0   